In [ ]:
!pip install -q PyPDF2 pdfplumber requests beautifulsoup4 sentence-transformers faiss-cpu tqdm
!pip install -q langchain chromadb pypdf python-docx unstructured requests
!pip install -q -U langchain-community fastapi uvicorn pyngrok nest_asyncio groq
!pip install -q pdfplumber sentence-transformers faiss-cpu beautifulsoup4 tqdm nest-asyncio
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, time, requests, nest_asyncio
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
from google.colab import files

# Dossiers
BASE_DIR = "/content/knowledge"
PDF_DIR = os.path.join(BASE_DIR, "pdfs")
CODE_DIR = os.path.join(BASE_DIR, "code")
WEB_DIR = os.path.join(BASE_DIR, "web")

for d in [BASE_DIR, PDF_DIR, CODE_DIR, WEB_DIR]:
    os.makedirs(d, exist_ok=True)


In [ ]:
uploaded = files.upload()
for name in uploaded.keys():
    if name.endswith(".pdf"):
        os.rename(name, os.path.join(PDF_DIR, name))
    else:
        os.rename(name, os.path.join(CODE_DIR, name))

print("Fichiers locaux ajoutés")




Saving 2412.08235v1.pdf to 2412.08235v1.pdf
Saving bsc_modelling_denning_sacco_ts.scala to bsc_modelling_denning_sacco_ts.scala
Saving bsc_modelling_denning_sacco_ts_with_attacker.scala to bsc_modelling_denning_sacco_ts_with_attacker.scala
Saving bsc_modelling_emv.scala to bsc_modelling_emv.scala
Saving bsc_modelling_emv_with_attacker.scala to bsc_modelling_emv_with_attacker.scala
Saving bsc_modelling_iso_9798_3.scala to bsc_modelling_iso_9798_3.scala
Saving bsc_modelling_iso_9798_3_with_attacker.scala to bsc_modelling_iso_9798_3_with_attacker.scala
Saving bsc_modelling_kerberos.scala to bsc_modelling_kerberos.scala
Saving bsc_modelling_kerberos_with_attacker.scala to bsc_modelling_kerberos_with_attacker.scala
Saving bsc_modelling_nsl.scala to bsc_modelling_nsl.scala
Saving bsc_modelling_nsl_with_attacker.scala to bsc_modelling_nsl_with_attacker.scala
Saving bsc_modelling_nssk.scala to bsc_modelling_nssk.scala
Saving bsc_modelling_nssk_with_attacker.scala to bsc_modelling_nssk_with_att

In [ ]:
!git clone -q https://github.com/UNamurCSFaculty/B2Scala.git /content/knowledge/gitB2Scala
print("Dépôt GitHub cloné")


Dépôt GitHub cloné


In [ ]:
DOC_URL = "https://staff.info.unamur.be/mbarkall/B2Scala/documentation/"
session = requests.Session()
session.headers.update({"User-Agent": "Colab-RAG/1.0"})

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def crawl(root, max_pages=30):
    seen, q, pages = set(), [root], []
    while q and len(pages) < max_pages:
        url = q.pop(0)
        if url in seen: continue
        seen.add(url)
        try:
            r = session.get(url, timeout=10, verify=False)
            if "text/html" in r.headers.get("Content-Type",""):
                soup = BeautifulSoup(r.text, "html.parser")
                for s in soup(["script", "style"]): s.decompose()
                text = soup.get_text(" ", strip=True)
                pages.append((url, text))
                for a in soup.find_all("a", href=True):
                    nxt = urljoin(url, a["href"])
                    if urlparse(nxt).netloc == urlparse(root).netloc and nxt not in seen:
                        q.append(nxt)
        except Exception as e:
            print("erreur:", e)
        time.sleep(0.3)
    return pages

html_pages = crawl(DOC_URL)
print("Pages doc récupérées :", len(html_pages))

# Sauvegarder les pages web
for i, (url, text) in enumerate(html_pages):
    filename = os.path.join(WEB_DIR, f"page_{i+1}.txt")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"URL: {url}\n\n{text}")
print("Pages web enregistrées dans 'knowledge/web'")


Pages doc récupérées : 30
Pages web enregistrées dans 'knowledge/web'


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, TextLoader, DirectoryLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader

documents = []

# PDF
pdf_loader = DirectoryLoader(PDF_DIR, glob="*.pdf", loader_cls=PyPDFLoader)
documents.extend(pdf_loader.load())

# TXT
txt_loader = DirectoryLoader(BASE_DIR, glob="**/*.txt", loader_cls=TextLoader)
documents.extend(txt_loader.load())

# Code B2Scala
code_loader = DirectoryLoader(CODE_DIR, glob="*.scala", loader_cls=TextLoader)
documents.extend(code_loader.load())

# Split en chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"Nombre total de chunks : {len(docs)}")


Nombre total de chunks : 952


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)
db.save_local("data/faiss_index")
print("Base de connaissance vectorielle créée et sauvegardée")


/tmp/ipython-input-150455016.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Base de connaissance vectorielle créée et sauvegardée


In [ ]:
query = "Donne-moi le code B2Scala pour le protocole Needham-Schroeder."
response = ask_b2scala(query)
print(response)


NameError: name 'ask_b2scala' is not defined

methodes  des metriques

In [ ]:
!pip install python-Levenshtein
!pip install codebleu

In [ ]:
pip install nltk pyrouge python-Levenshtein codebleu

In [ ]:
pip install --upgrade codebleu

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.chrf_score import sentence_chrf
import Levenshtein
#from codebleu import CodeBLEU  # Mise à jour de l'importation
from codebleu import codebleu

# Assurer que nltk est correctement initialisé
nltk.download('punkt')

# Fonction pour calculer BLEU
def calculate_bleu(reference, hypothesis):
    """
    Calcule la métrique BLEU entre une référence et une hypothèse.
    :param reference: Liste de tokens de la référence.
    :param hypothesis: Liste de tokens de l'hypothèse générée.
    :return: Score BLEU
    """
    return sentence_bleu([reference], hypothesis)

# Fonction pour calculer ChrF
def calculate_chrf(reference, hypothesis):
    """
    Calcule la métrique ChrF entre une référence et une hypothèse.
    :param reference: Liste de tokens de la référence.
    :param hypothesis: Liste de tokens de l'hypothèse générée.
    :return: Score ChrF
    """
    return sentence_chrf(reference, hypothesis)

# Fonction pour calculer la Distance d'édition
def calculate_edit_distance(reference, hypothesis):
    """
    Calcule la distance d'édition entre la référence et l'hypothèse générée.
    :param reference: Texte de référence.
    :param hypothesis: Texte de l'hypothèse générée.
    :return: Distance d'édition
    """
    return Levenshtein.distance(reference, hypothesis)

# Fonction pour calculer CodeBLEU
def calculate_codebleu(reference, hypothesis):
    """
    Calcule la métrique CodeBLEU entre une référence et une hypothèse.
    :param reference: Liste de tokens de la référence (généralement en format code).
    :param hypothesis: Liste de tokens de l'hypothèse générée (généralement en format code).
    :return: Score CodeBLEU
    """
    codebleu = codebleu()
    return codebleu.compute(reference, hypothesis)

# Exemple d'utilisation
def evaluate_code(reference_code, generated_code):
    """
    Calcule les métriques BLEU, ChrF, Edit Distance et CodeBLEU pour un exemple donné.
    :param reference_code: Code de référence (attendu).
    :param generated_code: Code généré par le modèle.
    :return: Dictionnaire des scores des métriques
    """
    # Préparation des textes sous forme de tokens
    reference_tokens = nltk.word_tokenize(reference_code)
    generated_tokens = nltk.word_tokenize(generated_code)

    # Calcul des métriques
    bleu_score = calculate_bleu(reference_tokens, generated_tokens)
    chrf_score = calculate_chrf(reference_code, generated_code)
    edit_distance = calculate_edit_distance(reference_code, generated_code)
    codebleu_score = calculate_codebleu(reference_code, generated_code)

    return {
        "BLEU": bleu_score,
        "ChrF": chrf_score,
        "EditDistance": edit_distance,
        "CodeBLEU": codebleu_score
    }

# Exemple de test
reference = """def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    less = [x for x in arr[1:] if x < pivot]
    greater = [x for x in arr[1:] if x >= pivot]
    return quicksort(less) + [pivot] + quicksort(greater)"""

generated = """def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    less = []
    greater = []
    for x in arr[1:]:
        if x < pivot:
            less.append(x)
        else:
            greater.append(x)
    return quicksort(less) + [pivot] + quicksort(greater)"""

# Évaluation du code
metrics = evaluate_code(reference, generated)

# Affichage des résultats
print("Évaluation des métriques du modèle:")
for metric, score in metrics.items():
    print(f"{metric}: {score}")


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.chrf_score import sentence_chrf
import Levenshtein
from codebleu import calc_codebleu  # Import correct pour CodeBLEU
import logging

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Téléchargements NLTK nécessaires
try:
    # Pour les nouvelles versions de NLTK
    nltk.download('punkt_tab', quiet=True)
except:
    # Fallback pour les anciennes versions
    nltk.download('punkt', quiet=True)

def calculate_bleu(reference, hypothesis):
    """
    Calcule la métrique BLEU entre une référence et une hypothèse.
    :param reference: Liste de tokens de la référence.
    :param hypothesis: Liste de tokens de l'hypothèse générée.
    :return: Score BLEU
    """
    if not reference or not hypothesis:
        return 0.0

    # BLEU attend une liste de références possibles
    return sentence_bleu([reference], hypothesis)

def calculate_chrf(reference, hypothesis):
    """
    Calcule la métrique ChrF entre une référence et une hypothèse.
    :param reference: Chaîne de caractères de référence.
    :param hypothesis: Chaîne de caractères de l'hypothèse générée.
    :return: Score ChrF
    """
    if not reference or not hypothesis:
        return 0.0

    # ChrF fonctionne sur des chaînes, pas des listes de tokens
    return sentence_chrf(reference, hypothesis)

def calculate_edit_distance(reference, hypothesis):
    """
    Calcule la distance d'édition entre la référence et l'hypothèse générée.
    :param reference: Texte de référence.
    :param hypothesis: Texte de l'hypothèse générée.
    :return: Distance d'édition normalisée
    """
    if not reference or not hypothesis:
        return float('inf') if reference != hypothesis else 0.0

    distance = Levenshtein.distance(reference, hypothesis)
    # Normalisation par la longueur maximale
    max_len = max(len(reference), len(hypothesis))
    return distance / max_len if max_len > 0 else 0.0

def calculate_codebleu(reference, hypothesis):
    """
    Calcule la métrique CodeBLEU entre une référence et une hypothèse.
    :param reference: Code de référence.
    :param hypothesis: Code généré.
    :return: Score CodeBLEU
    """
    if not reference or not hypothesis:
        return 0.0

    try:
        # CodeBLEU attend des listes de chaînes
        result = calc_codebleu([reference], [hypothesis], lang="python")
        return result['codebleu']
    except Exception as e:
        logger.warning(f"Erreur dans le calcul de CodeBLEU: {e}")
        return 0.0

def safe_tokenize(text):
    """Tokenisation sécurisée avec gestion d'erreurs"""
    try:
        return nltk.word_tokenize(text)
    except LookupError:
        # Fallback: tokenisation simple par espaces
        logger.warning("Tokenisation NLTK échouée, utilisation d'un fallback")
        return text.split()
    except Exception as e:
        logger.error(f"Erreur de tokenisation: {e}")
        return []

def evaluate_code(reference_code, generated_code):
    """
    Calcule les métriques BLEU, ChrF, Edit Distance et CodeBLEU pour un exemple donné.
    :param reference_code: Code de référence (attendu).
    :param generated_code: Code généré par le modèle.
    :return: Dictionnaire des scores des métriques
    """
    if not reference_code or not generated_code:
        return {
            "BLEU": 0.0,
            "ChrF": 0.0,
            "EditDistance": 1.0,
            "CodeBLEU": 0.0,
            "error": "Code vide fourni"
        }

    try:
        # Préparation des textes sous forme de tokens
        reference_tokens = safe_tokenize(reference_code.strip())
        generated_tokens = safe_tokenize(generated_code.strip())

        # Calcul des métriques avec gestion d'erreurs
        metrics = {}

        try:
            metrics["BLEU"] = calculate_bleu(reference_tokens, generated_tokens)
        except Exception as e:
            logger.error(f"Erreur BLEU: {e}")
            metrics["BLEU"] = 0.0

        try:
            metrics["ChrF"] = calculate_chrf(reference_code.strip(), generated_code.strip())
        except Exception as e:
            logger.error(f"Erreur ChrF: {e}")
            metrics["ChrF"] = 0.0

        try:
            metrics["EditDistance"] = calculate_edit_distance(reference_code.strip(), generated_code.strip())
        except Exception as e:
            logger.error(f"Erreur Edit Distance: {e}")
            metrics["EditDistance"] = 1.0

        try:
            metrics["CodeBLEU"] = calculate_codebleu(reference_code.strip(), generated_code.strip())
        except Exception as e:
            logger.error(f"Erreur CodeBLEU: {e}")
            metrics["CodeBLEU"] = 0.0

        return metrics

    except Exception as e:
        logger.error(f"Erreur générale dans evaluate_code: {e}")
        return {
            "BLEU": 0.0,
            "ChrF": 0.0,
            "EditDistance": 1.0,
            "CodeBLEU": 0.0,
            "error": str(e)
        }

def evaluate_b2scala_specific(reference_code, generated_code):
    """
    Évaluation spécifique pour B2Scala avec métriques additionnelles
    """
    basic_metrics = evaluate_code(reference_code, generated_code)

    # Métriques spécifiques à B2Scala
    b2scala_metrics = {
        "agent_count_match": count_agents_match(reference_code, generated_code),
        "message_structure_similarity": analyze_message_structure(reference_code, generated_code),
        "protocol_flow_accuracy": check_protocol_flow(reference_code, generated_code)
    }

    return {**basic_metrics, **b2scala_metrics}

def count_agents_match(ref, gen):
    """Vérifie si le nombre d'agents correspond"""
    ref_agents = len([line for line in ref.split('\n') if 'agent' in line.lower()])
    gen_agents = len([line for line in gen.split('\n') if 'agent' in line.lower()])
    return 1.0 if ref_agents == gen_agents else abs(ref_agents - gen_agents) / max(ref_agents, gen_agents, 1)

def analyze_message_structure(ref, gen):
    """Analyse la similarité de la structure des messages"""
    ref_messages = [line.strip() for line in ref.split('\n') if '->' in line]
    gen_messages = [line.strip() for line in gen.split('\n') if '->' in line]

    if not ref_messages and not gen_messages:
        return 1.0
    if not ref_messages or not gen_messages:
        return 0.0

    # Comparaison simple du nombre de messages
    return min(len(ref_messages), len(gen_messages)) / max(len(ref_messages), len(gen_messages))

def check_protocol_flow(ref, gen):
    """Vérifie la cohérence du flot du protocole"""
    # Implémentation simplifiée - à adapter selon vos besoins
    ref_steps = [line for line in ref.split('\n') if line.strip() and line[0].isdigit()]
    gen_steps = [line for line in gen.split('\n') if line.strip() and line[0].isdigit()]

    if not ref_steps and not gen_steps:
        return 1.0
    if not ref_steps or not gen_steps:
        return 0.0

    return min(len(ref_steps), len(gen_steps)) / max(len(ref_steps), len(gen_steps))

# Exemple d'utilisation avec gestion d'erreurs
if __name__ == "__main__":
    reference = """def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    less = [x for x in arr[1:] if x < pivot]
    greater = [x for x in arr[1:] if x >= pivot]
    return quicksort(less) + [pivot] + quicksort(greater)"""

    generated = """def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    less = []
    greater = []
    for x in arr[1:]:
        if x < pivot:
            less.append(x)
        else:
            greater.append(x)
    return quicksort(less) + [pivot] + quicksort(greater)"""

    # Évaluation du code
    try:
        metrics = evaluate_code(reference, generated)

        # Affichage des résultats
        print("Évaluation des métriques du modèle:")
        print("-" * 40)
        for metric, score in metrics.items():
            if metric != "error":
                if metric == "EditDistance":
                    print(f"{metric}: {score:.4f} (plus proche de 0 = mieux)")
                else:
                    print(f"{metric}: {score:.4f} (plus proche de 1 = mieux)")
            else:
                print(f"Erreur: {score}")

    except Exception as e:
        print(f"Erreur lors de l'évaluation: {e}")

    # Test avec du code B2Scala
    print("\n" + "="*50)
    print("Test avec code B2Scala:")

    b2scala_ref = """agent A, B : principal
var Na, Nb : nonce
var Ka, Kb : key

1. A -> B : {A, Na}Kb
2. B -> A : {B, Nb}Ka"""

    b2scala_gen = """agent A, B : principal
var Na, Nb : nonce
var Ka, Kb : key

1. A -> B : {A, Na}Kb
2. B -> A : {B, Nb, Na}Ka"""

    b2scala_metrics = evaluate_b2scala_specific(b2scala_ref, b2scala_gen)
    print("\nMétriques B2Scala:")
    print("-" * 40)
    for metric, score in b2scala_metrics.items():
        if isinstance(score, float):
            print(f"{metric}: {score:.4f}")
        else:
            print(f"{metric}: {score}")

In [ ]:
"""
Système d'évaluation corrigé pour B2Scala avec métriques améliorées
"""

# Installation des dépendances manquantes
"""
pip install tree-sitter-python tree-sitter-java tree-sitter-languages
pip install evaluate datasets transformers
pip install rouge-score sacrebleu
pip install bert-score
pip install textstat py-readability-metrics
pip install difflib
"""

import logging
import numpy as np
from typing import Dict, List, Tuple, Any
import re
import difflib
import ast
from collections import Counter
import warnings

# Suppression des warnings non critiques
warnings.filterwarnings("ignore", category=UserWarning)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_import_codebleu():
    """Import sécurisé de CodeBLEU avec installation automatique"""
    try:
        from codebleu import calc_codebleu
        return calc_codebleu, True
    except ImportError:
        logger.warning("CodeBLEU non disponible")
        return None, False

def safe_import_tree_sitter():
    """Import sécurisé de Tree-sitter"""
    try:
        import tree_sitter_python
        import tree_sitter_java
        return True
    except ImportError:
        logger.warning("Tree-sitter parsers non installés. Installez avec: pip install tree-sitter-python tree-sitter-java")
        return False

# Imports avec gestion d'erreurs
calc_codebleu_func, codebleu_available = safe_import_codebleu()
tree_sitter_available = safe_import_tree_sitter()

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.chrf_score import sentence_chrf
import Levenshtein

# Téléchargements NLTK sécurisés
try:
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
except:
    nltk.download('punkt', quiet=True)

class EnhancedCodeEvaluator:
    """Évaluateur de code amélioré avec métriques robustes"""

    def __init__(self):
        self.smoothing = SmoothingFunction().method4

    def calculate_bleu(self, reference: str, hypothesis: str) -> Dict[str, float]:
        """BLEU avec différents n-grams et lissage"""
        if not reference or not hypothesis:
            return {"bleu": 0.0, "bleu_1": 0.0, "bleu_2": 0.0, "bleu_3": 0.0, "bleu_4": 0.0}

        try:
            ref_tokens = nltk.word_tokenize(reference.lower())
            hyp_tokens = nltk.word_tokenize(hypothesis.lower())

            # BLEU avec lissage pour éviter les scores de 0
            bleu_score = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=self.smoothing)

            # BLEU individuel par n-gram
            bleu_1 = sentence_bleu([ref_tokens], hyp_tokens, weights=(1, 0, 0, 0), smoothing_function=self.smoothing)
            bleu_2 = sentence_bleu([ref_tokens], hyp_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=self.smoothing)
            bleu_3 = sentence_bleu([ref_tokens], hyp_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=self.smoothing)
            bleu_4 = sentence_bleu([ref_tokens], hyp_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=self.smoothing)

            return {
                "bleu": bleu_score,
                "bleu_1": bleu_1,
                "bleu_2": bleu_2,
                "bleu_3": bleu_3,
                "bleu_4": bleu_4
            }
        except Exception as e:
            logger.error(f"Erreur BLEU: {e}")
            return {"bleu": 0.0, "bleu_1": 0.0, "bleu_2": 0.0, "bleu_3": 0.0, "bleu_4": 0.0}

    def calculate_enhanced_chrf(self, reference: str, hypothesis: str) -> Dict[str, float]:
        """ChrF avec différentes variantes"""
        try:
            chrf_score = sentence_chrf(reference, hypothesis)
            chrf_word = sentence_chrf(reference, hypothesis, word_order=2)

            return {
                "chrf": chrf_score,
                "chrf_word": chrf_word
            }
        except Exception as e:
            logger.error(f"Erreur ChrF: {e}")
            return {"chrf": 0.0, "chrf_word": 0.0}

    def calculate_edit_metrics(self, reference: str, hypothesis: str) -> Dict[str, float]:
        """Métriques basées sur la distance d'édition"""
        if not reference or not hypothesis:
            return {"edit_distance": 1.0, "similarity": 0.0, "jaro_winkler": 0.0}

        try:
            # Distance d'édition normalisée
            distance = Levenshtein.distance(reference, hypothesis)
            max_len = max(len(reference), len(hypothesis))
            normalized_distance = distance / max_len if max_len > 0 else 0.0

            # Similarité basée sur la distance
            similarity = 1 - normalized_distance

            # Jaro-Winkler (meilleur pour les chaînes similaires)
            jaro_winkler = Levenshtein.jaro_winkler(reference, hypothesis)

            return {
                "edit_distance": normalized_distance,
                "similarity": similarity,
                "jaro_winkler": jaro_winkler
            }
        except Exception as e:
            logger.error(f"Erreur métriques d'édition: {e}")
            return {"edit_distance": 1.0, "similarity": 0.0, "jaro_winkler": 0.0}

    def calculate_codebleu_safe(self, reference: str, hypothesis: str, lang: str = "python") -> Dict[str, float]:
        """CodeBLEU avec gestion des erreurs et fallback"""
        if not codebleu_available or not tree_sitter_available:
            # Fallback: CodeBLEU simplifié basé sur les tokens
            return self._fallback_codebleu(reference, hypothesis)

        try:
            result = calc_codebleu_func([reference], [hypothesis], lang=lang)
            return {
                "codebleu": result.get('codebleu', 0.0),
                "ngram_match": result.get('ngram_match_score', 0.0),
                "weighted_ngram": result.get('weighted_ngram_match_score', 0.0),
                "syntax_match": result.get('syntax_match_score', 0.0),
                "dataflow_match": result.get('dataflow_match_score', 0.0)
            }
        except Exception as e:
            logger.warning(f"CodeBLEU échoué, utilisation du fallback: {e}")
            return self._fallback_codebleu(reference, hypothesis)

    def _fallback_codebleu(self, reference: str, hypothesis: str) -> Dict[str, float]:
        """CodeBLEU simplifié en cas d'échec"""
        try:
            # Analyse structurelle basique
            ref_structure = self._extract_code_structure(reference)
            hyp_structure = self._extract_code_structure(hypothesis)

            # Similarité structurelle
            structure_sim = self._calculate_structure_similarity(ref_structure, hyp_structure)

            # BLEU sur les tokens de code
            bleu_result = self.calculate_bleu(reference, hypothesis)

            return {
                "codebleu": (bleu_result["bleu"] + structure_sim) / 2,
                "ngram_match": bleu_result["bleu"],
                "weighted_ngram": bleu_result["bleu"],
                "syntax_match": structure_sim,
                "dataflow_match": structure_sim * 0.8  # Approximation
            }
        except Exception as e:
            logger.error(f"Erreur fallback CodeBLEU: {e}")
            return {"codebleu": 0.0, "ngram_match": 0.0, "weighted_ngram": 0.0,
                   "syntax_match": 0.0, "dataflow_match": 0.0}

    def _extract_code_structure(self, code: str) -> Dict[str, int]:
        """Extraction basique de la structure du code"""
        structure = {
            "lines": len(code.split('\n')),
            "words": len(code.split()),
            "braces": code.count('{') + code.count('}'),
            "arrows": code.count('->'),
            "agents": code.lower().count('agent'),
            "vars": code.lower().count('var'),
            "colons": code.count(':'),
            "numbers": len(re.findall(r'\d+\.', code))  # Numéros d'étapes
        }
        return structure

    def _calculate_structure_similarity(self, ref_struct: Dict, hyp_struct: Dict) -> float:
        """Calcule la similarité structurelle"""
        similarities = []
        for key in ref_struct:
            if key in hyp_struct:
                ref_val = ref_struct[key]
                hyp_val = hyp_struct[key]
                if ref_val == 0 and hyp_val == 0:
                    similarities.append(1.0)
                elif ref_val == 0 or hyp_val == 0:
                    similarities.append(0.0)
                else:
                    sim = min(ref_val, hyp_val) / max(ref_val, hyp_val)
                    similarities.append(sim)
        return np.mean(similarities) if similarities else 0.0

    def calculate_b2scala_specific_metrics(self, reference: str, hypothesis: str) -> Dict[str, float]:
        """Métriques spécifiques à B2Scala améliorées"""

        # Extraction des éléments B2Scala
        ref_elements = self._parse_b2scala_elements(reference)
        hyp_elements = self._parse_b2scala_elements(hypothesis)

        metrics = {}

        # Précision des agents
        agents_precision = self._calculate_agents_precision(ref_elements['agents'], hyp_elements['agents'])
        metrics['agents_precision'] = agents_precision

        # Précision des variables
        vars_precision = self._calculate_vars_precision(ref_elements['variables'], hyp_elements['variables'])
        metrics['variables_precision'] = vars_precision

        # Précision des messages
        messages_precision = self._calculate_messages_precision(ref_elements['messages'], hyp_elements['messages'])
        metrics['messages_precision'] = messages_precision

        # Précision du flux de protocole
        flow_precision = self._calculate_protocol_flow_precision(ref_elements['flow'], hyp_elements['flow'])
        metrics['protocol_flow_precision'] = flow_precision

        # Score B2Scala global
        b2scala_score = np.mean([agents_precision, vars_precision, messages_precision, flow_precision])
        metrics['b2scala_global_score'] = b2scala_score

        return metrics

    def _parse_b2scala_elements(self, code: str) -> Dict[str, List]:
        """Parse les éléments spécifiques à B2Scala"""
        elements = {
            'agents': [],
            'variables': [],
            'messages': [],
            'flow': []
        }

        lines = code.strip().split('\n')
        for line in lines:
            line = line.strip()
            if line.startswith('agent'):
                elements['agents'].extend(self._extract_agents(line))
            elif line.startswith('var'):
                elements['variables'].extend(self._extract_variables(line))
            elif '->' in line and any(c.isdigit() for c in line):
                elements['messages'].append(self._extract_message(line))
                elements['flow'].append(self._extract_flow_step(line))

        return elements

    def _extract_agents(self, line: str) -> List[str]:
        """Extrait les agents d'une ligne"""
        # Exemple: "agent A, B, S : principal"
        if ':' in line:
            agents_part = line.split(':')[0].replace('agent', '').strip()
            return [a.strip() for a in agents_part.split(',')]
        return []

    def _extract_variables(self, line: str) -> List[str]:
        """Extrait les variables d'une ligne"""
        # Exemple: "var Ka, Kb : key"
        if ':' in line:
            vars_part = line.split(':')[0].replace('var', '').strip()
            return [v.strip() for v in vars_part.split(',')]
        return []

    def _extract_message(self, line: str) -> Dict[str, str]:
        """Extrait les informations d'un message"""
        # Exemple: "1. A -> B : {A, Na}Kb"
        message_info = {'sender': '', 'receiver': '', 'content': ''}
        if '->' in line:
            parts = line.split('->')
            if len(parts) >= 2:
                sender_part = parts[0].strip()
                receiver_content = parts[1].strip()

                # Extraire le sender (enlever le numéro)
                sender = re.sub(r'^\d+\.?\s*', '', sender_part).strip()
                message_info['sender'] = sender

                # Extraire receiver et content
                if ':' in receiver_content:
                    receiver, content = receiver_content.split(':', 1)
                    message_info['receiver'] = receiver.strip()
                    message_info['content'] = content.strip()

        return message_info

    def _extract_flow_step(self, line: str) -> int:
        """Extrait le numéro d'étape du protocole"""
        match = re.match(r'^\s*(\d+)\.', line)
        return int(match.group(1)) if match else 0

    def _calculate_agents_precision(self, ref_agents: List, hyp_agents: List) -> float:
        """Calcule la précision des agents"""
        if not ref_agents and not hyp_agents:
            return 1.0
        if not ref_agents or not hyp_agents:
            return 0.0

        ref_set = set(ref_agents)
        hyp_set = set(hyp_agents)

        intersection = len(ref_set.intersection(hyp_set))
        union = len(ref_set.union(hyp_set))

        return intersection / union if union > 0 else 0.0

    def _calculate_vars_precision(self, ref_vars: List, hyp_vars: List) -> float:
        """Calcule la précision des variables"""
        return self._calculate_agents_precision(ref_vars, hyp_vars)  # Même logique

    def _calculate_messages_precision(self, ref_messages: List, hyp_messages: List) -> float:
        """Calcule la précision des messages"""
        if not ref_messages and not hyp_messages:
            return 1.0
        if not ref_messages or not hyp_messages:
            return 0.0

        # Comparaison structurelle des messages
        scores = []
        min_len = min(len(ref_messages), len(hyp_messages))

        for i in range(min_len):
            ref_msg = ref_messages[i]
            hyp_msg = hyp_messages[i]

            sender_match = 1.0 if ref_msg.get('sender') == hyp_msg.get('sender') else 0.0
            receiver_match = 1.0 if ref_msg.get('receiver') == hyp_msg.get('receiver') else 0.0

            # Similarité du contenu
            ref_content = ref_msg.get('content', '')
            hyp_content = hyp_msg.get('content', '')
            content_sim = Levenshtein.jaro_winkler(ref_content, hyp_content)

            msg_score = (sender_match + receiver_match + content_sim) / 3
            scores.append(msg_score)

        # Pénalité pour les messages manquants
        length_penalty = min_len / max(len(ref_messages), len(hyp_messages))

        return np.mean(scores) * length_penalty if scores else 0.0

    def _calculate_protocol_flow_precision(self, ref_flow: List, hyp_flow: List) -> float:
        """Calcule la précision du flux de protocole"""
        if not ref_flow and not hyp_flow:
            return 1.0
        if not ref_flow or not hyp_flow:
            return 0.0

        # Vérifier l'ordre des étapes
        ref_sorted = sorted(ref_flow)
        hyp_sorted = sorted(hyp_flow)

        # Correspondance exacte
        exact_match = 1.0 if ref_sorted == hyp_sorted else 0.0

        # Correspondance partielle
        common_steps = set(ref_flow).intersection(set(hyp_flow))
        partial_match = len(common_steps) / len(set(ref_flow).union(set(hyp_flow)))

        return (exact_match + partial_match) / 2

    def comprehensive_evaluate(self, reference: str, hypothesis: str, lang: str = "python") -> Dict[str, Any]:
        """Évaluation complète avec toutes les métriques"""
        results = {}

        # Métriques BLEU améliorées
        bleu_results = self.calculate_bleu(reference, hypothesis)
        results.update(bleu_results)

        # ChrF amélioré
        chrf_results = self.calculate_enhanced_chrf(reference, hypothesis)
        results.update(chrf_results)

        # Métriques d'édition
        edit_results = self.calculate_edit_metrics(reference, hypothesis)
        results.update(edit_results)

        # CodeBLEU sécurisé
        codebleu_results = self.calculate_codebleu_safe(reference, hypothesis, lang)
        results.update(codebleu_results)

        # Métriques spécifiques B2Scala
        b2scala_results = self.calculate_b2scala_specific_metrics(reference, hypothesis)
        results.update(b2scala_results)

        return results

def analyze_metrics_and_suggest_improvements(metrics: Dict[str, Any]) -> Dict[str, Any]:
    """Analyse les métriques et propose des améliorations"""

    analysis = {
        "score_interpretation": {},
        "strengths": [],
        "weaknesses": [],
        "improvement_suggestions": []
    }

    # Interprétation des scores
    bleu_score = metrics.get('bleu', 0)
    chrf_score = metrics.get('chrf', 0)
    edit_distance = metrics.get('edit_distance', 1)
    codebleu_score = metrics.get('codebleu', 0)
    b2scala_score = metrics.get('b2scala_global_score', 0)

    # Classification des performances
    def classify_score(score, higher_better=True):
        if higher_better:
            if score >= 0.9: return "Excellent"
            elif score >= 0.7: return "Bon"
            elif score >= 0.5: return "Moyen"
            elif score >= 0.3: return "Faible"
            else: return "Très faible"
        else:  # Pour edit_distance
            if score <= 0.1: return "Excellent"
            elif score <= 0.3: return "Bon"
            elif score <= 0.5: return "Moyen"
            elif score <= 0.7: return "Faible"
            else: return "Très faible"

    analysis["score_interpretation"] = {
        "BLEU": f"{bleu_score:.3f} ({classify_score(bleu_score)})",
        "ChrF": f"{chrf_score:.3f} ({classify_score(chrf_score)})",
        "Edit Distance": f"{edit_distance:.3f} ({classify_score(edit_distance, False)})",
        "CodeBLEU": f"{codebleu_score:.3f} ({classify_score(codebleu_score)})",
        "B2Scala Score": f"{b2scala_score:.3f} ({classify_score(b2scala_score)})"
    }

    # Forces identifiées
    if bleu_score >= 0.7:
        analysis["strengths"].append("Bonne correspondance lexicale (BLEU élevé)")
    if chrf_score >= 0.8:
        analysis["strengths"].append("Excellente similarité au niveau caractères (ChrF élevé)")
    if edit_distance <= 0.2:
        analysis["strengths"].append("Modifications minimales nécessaires")
    if b2scala_score >= 0.8:
        analysis["strengths"].append("Structure B2Scala bien préservée")

    # Faiblesses identifiées
    if bleu_score < 0.5:
        analysis["weaknesses"].append("Correspondance lexicale insuffisante")
    if codebleu_score < 0.3:
        analysis["weaknesses"].append("Structure syntaxique du code problématique")
    if edit_distance > 0.5:
        analysis["weaknesses"].append("Trop de modifications requises")
    if b2scala_score < 0.6:
        analysis["weaknesses"].append("Éléments B2Scala mal reproduits")

    # Suggestions d'amélioration
    if bleu_score < 0.7:
        analysis["improvement_suggestions"].append({
            "probleme": "Score BLEU faible",
            "solution": "Améliorer la correspondance des mots-clés B2Scala",
            "action": "Enrichir le corpus d'entraînement avec plus d'exemples B2Scala"
        })

    if codebleu_score < 0.5:
        analysis["improvement_suggestions"].append({
            "probleme": "CodeBLEU faible",
            "solution": "Améliorer la structure syntaxique du code généré",
            "action": "Utiliser des règles de grammaire B2Scala dans le prompt"
        })

    if edit_distance > 0.3:
        analysis["improvement_suggestions"].append({
            "probleme": "Distance d'édition élevée",
            "solution": "Réduire les différences inutiles",
            "action": "Post-traitement pour normaliser le format de sortie"
        })

    if metrics.get('agents_precision', 1) < 0.8:
        analysis["improvement_suggestions"].append({
            "probleme": "Agents mal identifiés",
            "solution": "Améliorer l'extraction des agents",
            "action": "Ajouter des exemples avec différents noms d'agents dans les données"
        })

    if metrics.get('protocol_flow_precision', 1) < 0.8:
        analysis["improvement_suggestions"].append({
            "probleme": "Flux de protocole incorrect",
            "solution": "Renforcer la logique de séquençage",
            "action": "Utiliser des contraintes temporelles dans la génération"
        })

    return analysis

# Test avec les données de l'utilisateur
if __name__ == "__main__":
    evaluator = EnhancedCodeEvaluator()

    # Test 1: Code Python
    print("=== Test 1: Code Python ===")
    reference_python = """def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    less = [x for x in arr[1:] if x < pivot]
    greater = [x for x in arr[1:] if x >= pivot]
    return quicksort(less) + [pivot] + quicksort(greater)"""

    generated_python = """def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    less = []
    greater = []
    for x in arr[1:]:
        if x < pivot:
            less.append(x)
        else:
            greater.append(x)
    return quicksort(less) + [pivot] + quicksort(greater)"""

    metrics_python = evaluator.comprehensive_evaluate(reference_python, generated_python)
    analysis_python = analyze_metrics_and_suggest_improvements(metrics_python)

    print("Métriques détaillées:")
    for metric, value in metrics_python.items():
        if isinstance(value, (int, float)):
            print(f"{metric}: {value:.4f}")

    print(f"\nAnalyse:")
    print("Interprétation des scores:", analysis_python["score_interpretation"])
    print("Forces:", analysis_python["strengths"])
    print("Faiblesses:", analysis_python["weaknesses"])
    print("Suggestions:", [s["solution"] for s in analysis_python["improvement_suggestions"]])

    # Test 2: Code B2Scala
    print("\n" + "="*50)
    print("=== Test 2: Code B2Scala ===")

    reference_b2scala = """agent A, B : principal
var Na, Nb : nonce
var Ka, Kb : key

1. A -> B : {A, Na}Kb
2. B -> A : {B, Nb}Ka"""

    generated_b2scala = """agent A, B : principal
var Na, Nb : nonce
var Ka, Kb : key

1. A -> B : {A, Na}Kb
2. B -> A : {B, Nb, Na}Ka"""

    metrics_b2scala = evaluator.comprehensive_evaluate(reference_b2scala, generated_b2scala, "java")
    analysis_b2scala = analyze_metrics_and_suggest_improvements(metrics_b2scala)

    print("Métriques détaillées:")
    for metric, value in metrics_b2scala.items():
        if isinstance(value, (int, float)):
            print(f"{metric}: {value:.4f}")

    print(f"\nAnalyse B2Scala:")
    print("Interprétation:", analysis_b2scala["score_interpretation"])
    print("Forces:", analysis_b2scala["strengths"])
    print("Faiblesses:", analysis_b2scala["weaknesses"])
    print("Suggestions:", [s["solution"] for s in analysis_b2scala["improvement_suggestions"]])